In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\Rizwan\\Desktop\\Courses\\E3E DS Project\\Data-Science-Project\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Rizwan\\Desktop\\Courses\\E3E DS Project\\Data-Science-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir : Path = Path
    source_URL: str = str
    local_data_file: Path = Path
    unzip_dir: Path = Path

In [17]:
from src.DataScienceProject.constants import *
from src.DataScienceProject.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


In [19]:
import os
import urllib.request as request
from src.DataScienceProject import logger
import zipfile

In [ ]:
## Component - data ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Downloading zip file
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-25 18:57:47,509: INFO: common: YAML file loaded successfully from config\config.yaml]
[2025-06-25 18:57:47,512: INFO: common: YAML file loaded successfully from params.yaml]
[2025-06-25 18:57:47,515: INFO: common: YAML file loaded successfully from schema.yaml]
[2025-06-25 18:57:47,516: INFO: common: Created directory: artifacts]
[2025-06-25 18:57:47,518: INFO: common: Created directory: artifacts/data_ingestion]
[2025-06-25 18:57:48,390: INFO: 2243478625: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9F4E:10F249:361A43B:3B654BC:685BF951
Accept-Ranges: bytes
Date: